In [1]:
!pip install antigranular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.6
    Uninstalling pydantic-2.10.6:
      Successfully uninstalled pydantic-2.10.6
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
wandb 0.19.8 requires py

In [2]:
import numpy as np
import pandas as pd

In [3]:
import sklearn as sk
from sklearn.ensemble import  RandomForestClassifier

In [4]:
import antigranular as an

In [5]:
session = an.login(<client_id>,<client_secret>,competition = "Fraud Detection Hackathon with Privacy Village #DPD25FEST")

Dataset "Transaction Fraud Hackathon Dataset" loaded to the kernel as transaction_fraud_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
test_x                         DataFrame      
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame

Connected to Antigranular server session id: fe70216e-eb46-41ae-9d69-5ee15941c6b4, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [6]:
%%ag
import op_opendp as dp
import op_snsynth  as op_snsynth
from op_opendp.mod import enable_features
enable_features("contrib")

/usr/local/lib/python3.10/site-packages/opdp/accuracy.py:224: DeprecationWarning: invalid escape sequence '\g'
  """Convert a discrete Laplacian scale into an accuracy estimate (tolerance) at a statistical significance level `alpha`.



In [7]:
%%ag
xtrain = transaction_fraud_hackathon_dataset['train_x']
ytrain = transaction_fraud_hackathon_dataset['train_y']
xtest = transaction_fraud_hackathon_dataset['test_x']

features = xtrain.columns
label = ytrain.columns

In [8]:
%%ag
ytrain.info()

+----+----------+-------------+---------------+---------+----------+
|    | Column   | numerical   | categorical   | dtype   | bounds   |
|----+----------+-------------+---------------+---------+----------|
|  0 | fraud    | True        | False         | float64 | (0, 1)   |
+----+----------+-------------+---------------+---------+----------+



In [9]:
%%ag
xtrain.info()

+----+----------+-------------+---------------+---------+----------+
|    | Column   | numerical   | categorical   | dtype   | bounds   |
|----+----------+-------------+---------------+---------+----------|
|  0 | a0       | True        | False         | float64 | (0, 1)   |
|  1 | a1       | True        | False         | float64 | (0, 1)   |
|  2 | a2       | True        | False         | float64 | (0, 1)   |
|  3 | a3       | True        | False         | float64 | (0, 1)   |
|  4 | a4       | True        | False         | float64 | (0, 1)   |
|  5 | a5       | True        | False         | float64 | (0, 1)   |
|  6 | a6       | True        | False         | float64 | (0, 1)   |
|  7 | a7       | True        | False         | float64 | (0, 1)   |
|  8 | a8       | True        | False         | float64 | (0, 1)   |
+----+----------+-------------+---------------+---------+----------+



In [10]:
%%ag
#### JOIN TRAIN DATA FEATUTRES AND TARGET LABEL
jnd_dataframe = xtrain.join(ytrain, how = 'inner')

In [11]:
%%ag
jnd_dataframe.info()

+----+----------+-------------+---------------+---------+----------+
|    | Column   | numerical   | categorical   | dtype   | bounds   |
|----+----------+-------------+---------------+---------+----------|
|  0 | a0       | True        | False         | float64 | (0, 1)   |
|  1 | a1       | True        | False         | float64 | (0, 1)   |
|  2 | a2       | True        | False         | float64 | (0, 1)   |
|  3 | a3       | True        | False         | float64 | (0, 1)   |
|  4 | a4       | True        | False         | float64 | (0, 1)   |
|  5 | a5       | True        | False         | float64 | (0, 1)   |
|  6 | a6       | True        | False         | float64 | (0, 1)   |
|  7 | a7       | True        | False         | float64 | (0, 1)   |
|  8 | a8       | True        | False         | float64 | (0, 1)   |
|  9 | fraud    | True        | False         | float64 | (0, 1)   |
+----+----------+-------------+---------------+---------+----------+



In [12]:
%%ag
#### FUNCTION MAPS FLOAT TYPE DATA TO INTEGER TYPE
def func(x: float) -> int:
    return 0 if x < 0.5 else 1

In [13]:
%%ag
#### MAP 'fraud' COLUMN  FROM FLOAT TYPE TO INTEGER TPE
jnd_dataframe['fraud'] = jnd_dataframe['fraud'].map( func, eps = 0.00001, output_bounds = (0,1))
jnd_dataframe.info()

+----+----------+-------------+---------------+---------+----------+
|    | Column   | numerical   | categorical   | dtype   | bounds   |
|----+----------+-------------+---------------+---------+----------|
|  0 | a0       | True        | False         | float64 | (0, 1)   |
|  1 | a1       | True        | False         | float64 | (0, 1)   |
|  2 | a2       | True        | False         | float64 | (0, 1)   |
|  3 | a3       | True        | False         | float64 | (0, 1)   |
|  4 | a4       | True        | False         | float64 | (0, 1)   |
|  5 | a5       | True        | False         | float64 | (0, 1)   |
|  6 | a6       | True        | False         | float64 | (0, 1)   |
|  7 | a7       | True        | False         | float64 | (0, 1)   |
|  8 | a8       | True        | False         | float64 | (0, 1)   |
|  9 | fraud    | True        | False         | int64   | (0, 1)   |
+----+----------+-------------+---------------+---------+----------+



In [14]:
%%ag
xtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a0      15000 non-null  float64
 1   a1      15000 non-null  float64
 2   a2      15000 non-null  float64
 3   a3      15000 non-null  float64
 4   a4      15000 non-null  float64
 5   a5      15000 non-null  float64
 6   a6      15000 non-null  float64
 7   a7      15000 non-null  float64
 8   a8      15000 non-null  float64
dtypes: float64(9)
memory usage: 1.0 MB



In [15]:
%%ag
### Add a column ('fraud') to the test data .. by adding this column we make the test data have the same number of coulmns as
### train data. This approach makes test data transformation smooth when the transformer is trained on train data.
xtest['fraud'] = 0
test = xtest

In [16]:
%%ag
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a0      15000 non-null  float64
 1   a1      15000 non-null  float64
 2   a2      15000 non-null  float64
 3   a3      15000 non-null  float64
 4   a4      15000 non-null  float64
 5   a5      15000 non-null  float64
 6   a6      15000 non-null  float64
 7   a7      15000 non-null  float64
 8   a8      15000 non-null  float64
 9   fraud   15000 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 1.1 MB



In [17]:
%%ag

#### DEFINING TRANSFORMATIONS ON INDIVIDUAL COLUMNS OF TRAIN DATA ,,
#### WANT USE StandardScaler FOR (a0 to a8) COLUMNS and MinMaxTransformer FOR ('fraud' column) TRANSFORMATION

import op_snsynth

eps = 0.02
mapping = {'a0': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a1': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a2': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a3': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a4': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a5': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a6': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a7': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'a8': op_snsynth.transform.StandardScaler(epsilon = eps, odometer = None),
        'fraud': op_snsynth.transform.MinMaxTransformer(lower = 0, upper = 1, negative = False, odometer = None),
        }

In [18]:
%%ag
#### CREATE DATA TRANSFORMER FROM PANDAS DATAFRAME
table_transformer = op_snsynth.transform.TableTransformer.from_pandas(jnd_dataframe, constraints = mapping)

In [19]:
%%ag
#### FIT AND TRANSFORM THE TRANSFORMER USING  TRAIN DATA
train_data = table_transformer.fit_transform(jnd_dataframe)

In [20]:
%%ag
#### TRANSFORM THE TEST DATA USING THE TRANSFORMER
test_data = table_transformer.transform(test)

In [21]:
%%ag
#### export train data feature names, target label
export(list(xtrain.columns), 'features')
export(list(ytrain.columns), 'label')

Setting up exported variable in local environment: features
Setting up exported variable in local environment: label


In [22]:
%%ag
### exporting transformed train data to user session
export(train_data, 'train_data')

Setting up exported variable in local environment: train_data


In [23]:
%%ag
### exporting transformed test data to user session
export(test_data, 'test_data')

Setting up exported variable in local environment: test_data


In [24]:
#### CHECK THE TYPE OF IMPORTED DATA
print( type(train_data),  type(test_data))

<class 'list'> <class 'list'>


In [25]:
###### create train dataframe from exported train data
label = ['fraud']
train_data = pd.DataFrame(train_data, columns = [features + label])

In [26]:
###### create  test dataframe from exported test data
test_data = pd.DataFrame(test_data, columns = [features + label])

In [27]:
#### DETERMINE 'fraud'  COUNT TRAIN DATA
print(train_data['fraud'].sum() , train_data['fraud'].count())

fraud    16626.0
dtype: float64 fraud    35000
dtype: int64


In [28]:
## SAVE TRAIN DATA
##train_data.to_csv('train_data.csv')

In [29]:
## SAVE TEST DATA
##test_data.to_csv('test_data.csv')

In [30]:
#### TRAIN DATA
Xtrain, Ytrain  = train_data.iloc[:,:9], train_data.iloc[:,9]

In [33]:
#### TEST DATA
XTest = test_data.iloc[:,:9]

In [34]:
#### DEFINING MODEL PARAMETERS AND MODEl
#### model parameters determined using optimization

params = {'n_estimators': 805, 'max_depth': 18}
model = RandomForestClassifier(**params, criterion = 'log_loss', class_weight ='balanced', random_state = 7)

In [35]:
#### TRAINING THE MODEL
model.fit(Xtrain, Ytrain)

RandomForestClassifier(class_weight='balanced', criterion='log_loss',
                       max_depth=18, n_estimators=805, random_state=7)

In [36]:
### MAKING PREDICTION ON XTest
YPred = model.predict(XTest)
YPred = pd.DataFrame(YPred.reshape(-1,1))

In [37]:
#### IMPORTING PREDICTION TO AG ENVIROMENT
session.private_import(YPred, 'YPrediction')

dataframe cached to server, loading to kernel...
DataFrame loaded successfully to the kernel



In [38]:
%%ag
#### CHECK THE UPLOADED PREDICTION DATA
YPrediction.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       15000 non-null  float64
dtypes: float64(1)
memory usage: 234.4 KB



In [39]:
%%ag
#### SUBMIT PREDICTION
submit_predictions(YPrediction)

error: Submission outside competition time window



In [40]:
#### CHECK PRIVACY COST
session.privacy_odometer()

{'total_epsilon_used': 0.0,
 'total_delta_used': 0.0,
 'library_costs': {'op_snsynth': {'total_delta': 0.0,
   'total_epsilon': 0.0,
   'total_requests': 4}},
 'dataset_costs': {}}

In [41]:
#### TERMINATE SESSION
session.terminate_session()

{'status': 'ok'}